In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [24]:
options = Options()
# options.add_argument('--headless')  # 필요 시 주석 해제
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

try:
    # URL 열기
    url = 'https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx?sort=HRA_RT'
    driver.get(url)

    # 드롭다운 메뉴에서 삼성 선택하기
    wait = WebDriverWait(driver, 10)
    
    # 팀 선택 드롭다운 메뉴 찾기
    team_dropdown = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlTeam_ddlTeam"]')))
    select = Select(team_dropdown)
    
    # 삼성 옵션 선택하기
    # KIA = 'HT' 삼성 = 'SS" LG = 'LG' 두산 = 'OB' KT = 'KT' SSG = 'SK' 한화 = 'HH' 롯데 = 'LT' NC = 'NC' 키움 = 'WO'
    select.select_by_value('SS')

    # 선택 후 페이지 갱신 대기
    time.sleep(5)  # 페이지 로드 대기 시간 (조정 가능)
    
    # 페이지 소스 가져오기
    html = driver.page_source

    # BeautifulSoup으로 파싱하기
    soup = BeautifulSoup(html, 'html.parser')
   
finally:
    # 웹 드라이버 종료
    driver.quit()

table = soup.find('table', class_="tData01 tt")
thead = table.thead

head_list=[]
for col in thead.find_all('th'):
    head_list.append(col.text)

tbody = soup.find('tbody')
data = []
rank = 1

for tr in tbody.find_all('tr'):
    row = [rank]  # Add rank information

    # Collect cell data
    for col in tr.find_all('td'):
        row.append(col.get_text(strip=True))

    data.append(row)
    rank += 1

# Remove the rank column from each row
data = [row[1:] for row in data]

df = pd.DataFrame(data=data, columns=head_list)
column_mapping = {
    'AVG': '타율',
    'G': '경기',
    'PA': '타석',
    'AB': '타수',
    'R': '득점',
    'H': '안타',
    '2B': '2루타',
    '3B': '3루타',
    'HR': '홈런',
    'TB': '루타',
    'RBI': '타점',
    'SAC': '희생번트',
    'SF' : '희생플라이'
}
df.rename(columns=column_mapping, inplace=True)
df.set_index('순위', inplace=True)
df

,선수명,팀명,타율,경기,타석,타수,득점,안타,2루타,3루타,홈런,루타,타점,희생번트,희생플라이
순위,,,,,,,,,,,,,,,
1,김민수,삼성,1.000,1,1,1,1,1,0,0,1,4,3,0,0
2,디아즈,삼성,0.333,7,29,27,6,9,1,0,3,19,7,0,0
2,카데나스,삼성,0.333,7,25,24,2,8,2,0,2,16,5,0,0
4,구자욱,삼성,0.318,109,481,421,69,134,34,1,22,236,84,2,5
5,김지찬,삼성,0.311,118,463,392,84,122,15,0,3,146,27,12,1
6,윤정빈,삼성,0.310,50,147,126,21,39,4,1,5,60,17,0,2
7,강민호,삼성,0.307,119,402,355,43,109,15,1,17,177,72,0,5
8,맥키넌,삼성,0.294,72,315,272,28,80,13,0,4,105,36,0,3
9,김헌곤,삼성,0.289,97,263,239,35,69,7,0,8,100,27,3,0
